In [13]:
# imports
import sys, os
sys.path.append(os.getcwd()+'/code')
import 
import numpy as np
import pickle as pkl
import Algorithms
from data_streamer import Streamer, MultiStreamer

In [14]:
# constants
train_data_file=os.getcwd() + '/data/train_mnist.csv'
validation_data_file=os.getcwd() + '/data/test_mnist.csv'
assert os.path.isfile(train_data_file)
k = 100 # dimensionality of representations
d = 784 # dimensionality of input
learning_rate = 1.0 # initial learning rate
validation_freq = 1000 # how many training steps in between validations

In [15]:
# data streamer and algorithm
train_stream = Streamer(train_data_file).get_stream()
algorithm=Algorithms.oja({'d': d, 'k': k, 'learning_rate': learning_rate})
# load validation data at once to make validation faster
validation_data=[]
for point in Streamer(validation_data_file).get_stream():
    validation_data.append(point)
validation_data=np.stack(validation_data,axis=0)

In [16]:
# placeholders for loss
validation_loss, training_loss, min_training_loss = [], [], []
training_batch = []
# main loop
counter=0
for point in train_stream:
    counter+=1
    algorithm.step(point)
    training_batch.append(point)
    if counter%validation_freq==0:
        loss=algorithm.loss(validation_data)
        loss/=(validation_data.shape[0]*validation_data.shape[1])
        validation_loss.append(loss)
        training_batch=np.stack(training_batch,axis=0)
        loss=algorithm.loss(training_batch)
        loss/=(training_batch.shape[0]*training_batch.shape[1])
        training_loss.append(loss)
        # for information, compute the los for the non-stochastic, closed form solution
        # our training loss couldn't be lower than that
        lambdas,_=np.linalg.eigh(np.matmul(training_batch.T,training_batch))
        loss=sum(lambdas[:-k])/(training_batch.shape[0]*training_batch.shape[1])
        min_training_loss.append(loss) 
        training_batch=[]

In [17]:
# plot the train and test loss
plt.figure(figsize=(8, 8))
plt.xlabel('Step')
plt.ylabel('Loss')
plt.yscale(YSCALE)
x_scale=[x*validation_freq for x in range(1,len(training_loss)+1)]
plt.plot(x_scale, training_loss, '--b', label='train loss')
plt.plot(x_scale, validation_loss, '--r', label='test loss')
plt.plot(x_scale, min_training_loss, '--k', label='train loss lower bound')
_ = plt.legend()
plt.savefig('here.pdf')

NameError: name 'plt' is not defined